In [47]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Conv1D,
    LSTM,
    Bidirectional,
    Dropout,
    BatchNormalization,
    Dense,
    MaxPooling1D,
    concatenate,
    GlobalAveragePooling1D,
    Flatten,
)
from sklearn.metrics import confusion_matrix
from keras.models import save_model, load_model
import joblib
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [48]:
df = pd.read_csv("audio_features.csv")
df.head()

,chroma_stft,rms,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,...,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,origin_sample,LABEL
0,0.369246,0.114583,2329.765047,1668.270744,4210.475297,0.125022,-344.44583,74.368630,-26.999317,38.491272,...,-0.012907,-3.529766,-4.690998,-13.813165,-6.292424,-10.091806,-1.323195,-8.031024,LA_T_6747050,fake
1,0.261991,0.172123,1387.932080,1209.900252,2261.231024,0.083097,-257.60925,129.847320,-36.567460,33.844730,...,-12.913269,-7.028874,-6.108995,-24.280165,-14.908502,-14.017176,-6.970436,-15.313593,LA_T_6747050,fake
2,0.367695,0.070103,2301.121569,1604.363941,3903.382457,0.135931,-322.69806,86.057620,-30.443005,41.763557,...,5.834561,-1.498449,1.390241,-17.747652,-10.772357,-9.536011,-3.103892,-11.773501,LA_T_6747050,fake
3,0.630570,0.000076,3492.844688,2376.639078,6383.248901,0.171265,-786.05550,56.160923,-37.678963,50.892410,...,12.214974,-3.049933,13.535940,-7.106358,11.727142,-6.619684,5.530129,-3.223867,LA_T_6747050,fake
4,0.417924,0.091955,2470.647169,1780.860052,4330.865479,0.177080,-295.70105,67.723100,-19.865486,33.981766,...,3.750639,-6.569212,5.015835,-8.746722,-2.513248,-10.342720,1.080411,-9.256080,LA_T_7031111,fake


In [49]:
X = df.iloc[:, :-2]
y = df.iloc[:, -1]

In [50]:
lb = preprocessing.LabelBinarizer()
y = lb.fit_transform(y)
y = y.ravel()
print(y)

[0 0 0 ... 1 1 1]


In [51]:
X_train,X_Test,Y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)
X_train.shape[1]

26

In [52]:
scaler = StandardScaler()
scaler.fit(X_train)
joblib.dump(scaler, "scaler_1.joblib")
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_Test)

X_train_std = X_train_std.reshape((X_train_std.shape[0], X_train_std.shape[1], 1))
X_test_std = X_test_std.reshape((X_test_std.shape[0], X_test_std.shape[1], 1))


joblib.dump(X_train_std, "X_train_std.joblib")
joblib.dump(X_test_std, "X_test_std.joblib")

['X_test_std.joblib']

In [53]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


def WIRENetSpoofImprovedEnhanced(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, 3, activation="relu", padding="same")(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(128, 3, activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(256, 3, activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.5)(x)  # Increased dropout rate for regularization

    x = Conv1D(512, 3, activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.5)(x)

    x = Conv1D(64, 3, activation="relu", padding="same")(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(128, 3, activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(256, 3, activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Dropout(0.5)(x)

    x = Conv1D(512, 3, activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)

    x = GlobalAveragePooling1D()(x)
    x = Dense(256, activation="relu", kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


input_shape = (X_train_std.shape[1], 1)


def lr_schedule(epoch, lr):
    if epoch > 0 and epoch % 10 == 0:
        return lr * 0.1
    return lr


# callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

model = WIRENetSpoofImprovedEnhanced(input_shape)

model.compile(
    optimizer=Adam(learning_rate=0.000001),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

model.fit(
    X_train_std,
    Y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test_std, y_test)
)

Epoch 1/10
2489/2489 [==============================] - 80s 30ms/step - loss: 4.1981 - accuracy: 0.5139 - val_loss: 3.8462 - val_accuracy: 0.8285
Epoch 2/10
2489/2489 [==============================] - 3323s 1s/step - loss: 3.9472 - accuracy: 0.6374 - val_loss: 3.6553 - val_accuracy: 0.8898
Epoch 3/10
2489/2489 [==============================] - 50s 20ms/step - loss: 3.7481 - accuracy: 0.7300 - val_loss: 3.5023 - val_accuracy: 0.8981
Epoch 4/10
2489/2489 [==============================] - 55s 22ms/step - loss: 3.5765 - accuracy: 0.7987 - val_loss: 3.3822 - val_accuracy: 0.8989
Epoch 5/10
2489/2489 [==============================] - 57s 23ms/step - loss: 3.4371 - accuracy: 0.8386 - val_loss: 3.2795 - val_accuracy: 0.8991
Epoch 6/10
2489/2489 [==============================] - 60s 24ms/step - loss: 3.3130 - accuracy: 0.8605 - val_loss: 3.1835 - val_accuracy: 0.8991
Epoch 7/10
2489/2489 [==============================] - 78s 31ms/step - loss: 3.2055 - accuracy: 0.8711 - val_loss: 3.0903 -

In [54]:
model.save("wirenet_model.h5")

# Load the model
loaded_model = load_model("wirenet_model.h5")

c:\Users\manis\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [55]:
test_df = pd.read_csv("audio_features_eval.csv")
X_test = test_df.drop(["origin_sample", "LABEL"], axis=1)
y_true = test_df["LABEL"]

# Encode the labels
y_true_encoded = lb.fit_transform(y_true)

# Standardize the features
scaler = StandardScaler()
X_test_std = scaler.fit_transform(X_test)
X_test_std = X_test_std.reshape((X_test_std.shape[0], X_test_std.shape[1], 1))

# Make predictions with your model
predictions = model.predict(X_test_std)
predictions_label = (predictions > 0.5).astype("int32")

# Calculate accuracy using the encoded labels
accuracy = accuracy_score(y_true_encoded, predictions_label)
print(f"Accuracy: {accuracy * 100:.2f}%")

3319/3319 [==============================] - 24s 7ms/step
Accuracy: 88.62%
